In [1]:
with open("input22.txt", "r") as f:
    f = f.read().strip()
data = f.split("\n")

In [2]:
print(data)

['depth: 11394', 'target: 7,701']


In [3]:
depth = int(data[0].split()[1])
target = tuple(map(int,data[1].split()[1].split(",")))

In [4]:
import numpy as np
arr = np.zeros((max(target)*2,max(target)*2),dtype="int32")

In [5]:
for row in range(arr.shape[0]):
    for col in range(arr.shape[1]):
        if (row,col)==(0,0) or (col,row)==target:
            geologic_index = 0
        elif row==0:
            geologic_index = col * 16807
        elif col==0:
            geologic_index = row * 48271
        else:
            geologic_index = arr[row-1,col] * arr[row,col-1]
        erosion = (geologic_index + depth) % 20183
        arr[row,col] = erosion

In [6]:
arr_types = arr%3
print(np.sum(arr_types[:target[1]+1,:target[0]+1]))

5637


In [17]:
regions = {0:["torch", "gear"],
           1:["gear", "nothing"],
           2:["torch", "nothing"]
        }
common_tools = {}
for r1 in range(3):
    for r2 in range(3):
        common = [x for x in regions[r1] if x in regions[r2]]
        common_tools[(r1,r2)] = common
print(common_tools)

{(0, 1): ['gear'], (1, 2): ['nothing'], (0, 0): ['torch', 'gear'], (2, 0): ['torch'], (1, 0): ['gear'], (2, 2): ['torch', 'nothing'], (0, 2): ['torch'], (2, 1): ['nothing'], (1, 1): ['gear', 'nothing']}


In [8]:
from collections import defaultdict

In [19]:
best_times = defaultdict(lambda: {"torch":9999999, "gear":9999999, "nothing":9999999})
best_times[(0,0)]["torch"] = 0

available_locs = {(0,0)}
while available_locs:

    #Save this each loop so we can halt before filling in entire array
    best_time = best_times[target[::-1]]["torch"]
    
    new_locs = set()
    
    for loc in available_locs:
        #Look for cheaper move in current spot
        region = arr_types[loc]
        tool0, tool1 = regions[region]
        
        if best_times[loc][tool0] + 7 < best_times[loc][tool1]:
            best_times[loc][tool1] = best_times[loc][tool0] + 7
            new_locs.add(loc)
        if best_times[loc][tool1] + 7 < best_times[loc][tool0]:
            best_times[loc][tool0] = best_times[loc][tool1] + 7
            new_locs.add(loc)

        #Look for cheaper move in surrounding spots
        moves = [(loc[0]-1, loc[1]), (loc[0]+1, loc[1]), (loc[0],loc[1]-1), (loc[0],loc[1]+1)]
        moves = [x for x in moves if x[0]>=0 and x[0]<arr_types.shape[0] and x[1]>=0 and x[1]<arr_types.shape[1]]
        for new_loc in moves:

            new_region = arr_types[new_loc]
            transition_tools = common_tools[(region, new_region)]
            
            for t in transition_tools:
                if best_times[loc][t] + 1 < best_times[new_loc][t]:
                    best_times[new_loc][t] = best_times[loc][t] + 1
                    #Only continue if possible improvement on our best time
                    if min(best_times[new_loc].values()) < best_time:
                        new_locs.add(new_loc)

    available_locs = new_locs

print(best_time)

969
